In [6]:
import datetime
import pandas as pd
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import requests
from selenium.webdriver.common.by import By
import re
import os
import warnings
from pyvi import ViTokenizer, ViPosTagger
from fuzzywuzzy import fuzz
import pandas as pd
import requests
from io import BytesIO
import easyocr
from PIL import Image
import urllib.request
warnings.filterwarnings("ignore")

c:\Users\dangt\OneDrive\Máy tính\analysis_breaking_news\.venv\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


**Crawl data**

1. Crawl thể thao văn hóa

In [7]:
def crawl_category_thethaovanhoa():
    options = Options()
    options.add_argument('--headless')
    driver = webdriver.Chrome(options=options)
    driver.get("https://thethaovanhoa.vn/")
    
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    ul_element = soup.find("ul", class_="wrapper")
    li_elements = ul_element.find_all("li", class_=True)
    
    category = {}
    for li in li_elements:
        a_element = li.find("a")
        title = a_element.get("title")
        if title is not None:
            title = a_element["title"]
            href = a_element["href"]
            category[title] = href
        
    driver.quit()
    return category

In [8]:
def crawl_content_thethaovanhoa(link):
    response = requests.get(f'https://thethaovanhoa.vn{link}')
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    p_time = soup.find('p', class_='news-time left')
    if p_time is not None:
        time = p_time.find('span').text.strip()
    else:
        time = None

    div_content = soup.find('div', class_='entry-body normal clearafter')
    if div_content is not None:
        b_element = div_content.find('b')
        texts = [element.get_text(strip=True) for element in div_content.find_all(['h2', 'p'])]
        text_str = '\n'.join(texts)
        if b_element is not None:
            summary = b_element.text
            return time, summary, text_str
        else:
            return time, None, text_str

    return None, None, None

In [9]:
def crawl_thethaovanhoa(file_name, category):
    filename = f'./data/raw_data/thethaovanhoa/{file_name}.csv'
    fields = ['time', 'category', 'title', 'link', 'summary', 'content']
    categories = list(category.keys())
    hrefs = list(category.values())
    
    with open(filename, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fields)
        for i in range(1, len(category)-1):
            href = hrefs[i]
            link = f'https://thethaovanhoa.vn{href}'
            response = requests.get(link)
            html = response.text
            soup = BeautifulSoup(html, 'html.parser')
            
            div_first_content = soup.find('div', class_='content left')
            h2 = div_first_content.find('h2')
            link = h2.find('a')
            href = link.get('href')
            title = link.get('title').strip()
            time, summary, content = crawl_content_thethaovanhoa(href)
            writer.writerow({'time': time, 'summary': summary, 'content': content, 'category': categories[i], 'title': title, 'link': f'https://thethaovanhoa.vn{href}'})
            
            box_highlight_list = soup.find('ul', class_='box-highlight-list')
            li_elements = box_highlight_list.find_all('li')
            for li in li_elements:
                h3_element = li.find('h3', class_='title')
                title = h3_element.text
                href = h3_element.a['href']
                time, summary, content = crawl_content_thethaovanhoa(href)
                writer.writerow({'time': time, 'summary': summary, 'content': content, 'category': categories[i], 'title': title, 'link': f'https://thethaovanhoa.vn{href}'})
            
            ul_element = soup.find("ul", class_="news-stream col550 left col547")
            if ul_element is not None:
                div_content_elements = ul_element.find_all("div", class_="left text")
                for div in div_content_elements:
                    h3_element = div.find('h3', class_='title')
                    title = h3_element.text
                    href = h3_element.a['href']
                    time, summary, content = crawl_content_thethaovanhoa(href)
                    writer.writerow({'time': time, 'summary': summary, 'content': content, 'category': categories[i], 'title': title, 'link': f'https://thethaovanhoa.vn{href}'})

2. Crawl vietnamplus

In [10]:
def check_time(time_string):
    current_date = datetime.datetime.now().date()
    
    if re.match(r"\d{2}/\d{2}/\d{4}", time_string[:10]):
        target_date = datetime.datetime.strptime(time_string[:10], "%d/%m/%Y").date()
        difference = (current_date - target_date).days

        if difference == 0:
            return 1
        else:
            return 0
    return 1

In [11]:
def crawl_content_vietnamplus(link):
    response = requests.get(f'https://www.vietnamplus.vn{link}', verify=False)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    
    if soup.find('time') is not None:
        time = soup.find('time').text.strip()
    else:
        time = None
    
    if soup.find('time') is not None:
        summary = soup.find('div', class_='details__summary cms-desc').text.strip()
    else:
        summary = None
    
    contents = soup.find_all('p')
    if contents is not None:
        texts = [content.get_text(strip=True) for content in contents]
        text_str = '\n'.join(texts)
    else:
        text_str = ''
    
    return time, summary, text_str

In [12]:
def crawl_vietnamplus(file_name, category):
    filename = f'./data/raw_data/vietnamplus/{file_name}.csv'
    fields = ['time', 'category', 'title', 'link', 'summary', 'content']
    categories = list(category.keys())
    hrefs = list(category.values())
    
    with open(filename, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fields)
        for i in range(len(category)):
            for j in range(1,500):
                href = hrefs[i]
                href = href.split(".")[0]
                link = f'https://www.vietnamplus.vn{href}/trang{j}.vnp'
                response = requests.get(link, verify=False)
                html = response.text
                soup = BeautifulSoup(html, 'html.parser')
                
                div_first_content = soup.find('article', class_='story--large')
                if div_first_content is not None:
                    title = div_first_content.find('h2').text.strip()
                    link = div_first_content.find('h2').find('a')
                    href = link.get('href')
                    time, summary, content = crawl_content_vietnamplus(href)
                    writer.writerow({'time': time, 'summary': summary, 'content': content, 'category': categories[i], 'title': title, 'link': f'https://www.vietnamplus.vn{href}'})
                
                div_cate = soup.find('div', class_ = 'cate__focus')
                if div_cate is not None:
                    div_second_content = div_cate.find_all('article', class_="story")
                    if div_second_content is not None:
                        for div in div_second_content:
                            a_tag = div.find('a', class_="story__title cms-link")
                            title = a_tag.text.strip()
                            link = a_tag.get('href')
                            time, summary, content = crawl_content_vietnamplus(link)
                            writer.writerow({'time': time, 'summary': summary, 'content': content, 'category': categories[i], 'title': title, 'link': f'https://www.vietnamplus.vn{link}'})
                
                div_third_content = soup.find_all('article', class_="story story--split")
                if div_third_content is not None:
                    for div in div_third_content:
                        h2 = div.find('h2').find('a', class_="story__title cms-link")
                        title = h2.text.strip()
                        link = h2.get('href')
                        time, summary, content = crawl_content_vietnamplus(link)
                        writer.writerow({'time': time, 'summary': summary, 'content': content, 'category': categories[i], 'title': title, 'link': f'https://www.vietnamplus.vn{link}'})
                
                div_content = soup.find('div', class_ = 'clearfix')
                articles = div_content.find_all('article', class_ = 'story story--split')
                if articles is not None:
                    for div in articles:
                        h2 = div.find('h2').find('a', class_="story__title cms-link")
                        title = h2.text.strip()
                        link = h2.get('href')
                        time, summary, content = crawl_content_vietnamplus(link)
                        if (time != None):
                            last_time = time
                        writer.writerow({'time': time, 'summary': summary, 'content': content, 'category': categories[i], 'title': title, 'link': f'https://www.vietnamplus.vn{link}'})
                
                # print(last_time)
                if (check_time(last_time) == 0):
                    break
            

3. Crawl Thông tấn xã Việt Nam

In [13]:
def convert_time(time_string):
    
    current_date = datetime.datetime.now().date()
    
    target_date = datetime.datetime.strptime(time_string[:10], "%d/%m/%Y").date()
    # print(target_date)
    difference = (current_date - target_date).days

    if difference <= 30:
        return 1
    else:
        return 0

In [14]:
def crawl_ttxvn(file_name, category):
    filename = f'./data/raw_data/thongtanxavn/{file_name}.csv'
    fields = ['news source', 'time', 'category', 'title', 'link', 'summary']
    categories = list(category.keys())
    hrefs = list(category.values())
    
    with open(filename, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fields)
        for i in range(1, len(category)):
            href = hrefs[i]
            link = f'http://vnanet.vn{href}'
            response = requests.get(link)
            html = response.text
            soup = BeautifulSoup(html, 'html.parser')
            
            
            div = soup.find('div', class_ = 'feature-list-news')
            if div is not None:
                ul = div.find('ul')
                li_elements = ul.find_all('li')
                for li in li_elements:
                    div_content = li.find('div', class_ = "grp-panel")
                    a = div_content.find('a')
                    href = a.get('href')
                    summary_p = div_content.find('p', class_='sample-text-ft')
                    if summary_p is not None:
                        summary = summary_p.text.strip()
                    else:
                        summary = ""
                    title = a.text
                    div_time = div_content.find('div', class_ = "panel-rows")
                    time = div_time.find('span').text.strip()
                    # print(time)
                    news_source = li.select_one('.partner-name').text
                    if (news_source != 'VietnamPlus'):
                        writer.writerow({'news source': news_source, 'time': time, 'summary': summary, 'category': categories[i], 'title': title, 'link': href})

            for j in range(1, 300):
                href = hrefs[i]
                link = f'http://vnanet.vn{href}page/{j}.html'
                response = requests.get(link)
                html = response.text
                soup = BeautifulSoup(html, 'html.parser')
                div2 = soup.find('div', class_ = "grp-list-news-2")
                if div2 is not None:
                    ul2 = div2.find('ul')
                    li2_elements = ul2.find_all('li')
                    for li in li2_elements:
                        div_content = li.find('div', class_ = "grp-panel")
                        a = div_content.find('a', target="_blank")
                        href = a.get('href')
                        summary_p = div_content.find('p', class_='sample-text-ft')
                        if summary_p is not None:
                            summary = summary_p.text.strip()
                        title = a.text
                        div_time = div_content.find('div', class_ = "panel-rows")
                        time_element = div_time.find_all('span')
                        for x in time_element:
                            time_check = x.text.strip()
                            if time_check != 'VietnamPlus':
                                time_info = time_check
                        news_source = li.select_one('.partner-name').text.strip()
                        if (news_source != 'VietnamPlus'):
                            writer.writerow({'news source': news_source, 'time': time_info, 'summary': summary, 'category': categories[i], 'title': title, 'link': href})
                if (check_time(time_check) == 0):
                    break

In [15]:
category_vnplus = {'Chính trị': '/chinhtri.vnp', 'Thế giới ': '/thegioi.vnp', 'Kinh tế ': '/kinhte.vnp', 'Xã hội ': '/xahoi.vnp', 'Đời sống ': '/doisong.vnp', 
                   'Văn hóa ': '/vanhoa.vnp', 'Thể thao ': '/thethao.vnp', 'World Cup 2022': '/worldcup2022.vnp',
                   'Khoa học ': '/khoahoc.vnp', 'Công nghệ ': '/congnghe.vnp', 
                   'Ôtô-Xe máy': '/otoxemay.vnp', 'Môi trường': '/moitruong.vnp', 'Du lịch ': '/dulich.vnp', 
                   'Thị trường': '/tinthitruong.vnp', 'Chuyện lạ': '/chuyenla.vnp', 'RapNewsPlus': '/rapnewsplus.vnp', 'News Game ': '/newsgame.vnp', 
                   'Game thời sự': '/gamenews.vnp', 'Game giải trí': '/gamefun.vnp', 'Game kiến thức': '/gameeducation.vnp', 'Thăm dò ý kiến': '/gamepoll.vnp', 
                   'Podcast': '/podcast.vnp', 'Tin ảnh': '/photo/', 'Video': '/video/', 
                   'Infographics': '/infographics.vnp', 'Timeline': '/timeline.vnp', 'Tổng hợp': '/topicnews.vnp', 'Ảnh 360': '/photo360.vnp', 
                   'Mega Story': '/megastory.vnp'}

In [16]:
category_thethaovanhoa = {'Trang chủ': '/', 'Bóng đá Việt': '/bong-da-viet-nam.htm', 'Bóng đá Quốc tế': '/bong-da-anh.htm', 
                          'Nhận định bóng đá': '/du-doan-bong-da.htm', 'Thể thao': '/the-thao.htm', 'Thế giới Sao': '/the-gioi-sao.htm', 
                          'Văn hoá': '/van-hoa.htm', 'Giải trí': '/giai-tri.htm', 'Kbiz': '/kbiz-kpop-phim-han.htm', 'Đời sống': '/doi-song.htm', 
                          'Tin tức 24h': '/tin-tuc-24h.htm', 'HighTech': '/hightech.htm', 'Bạn cần biết': '/ban-can-biet.htm', 'Multimedia ': '/multimedia.htm', 
                          'Video': '/video.htm'}

In [17]:
category_ttxvn = {'Trang chủ': '/vi/', 'Chính trị': '/vi/tin-tuc/chinh-tri-11/', 'Xã hội': '/vi/tin-tuc/xa-hoi-14/', 'Kinh tế': '/vi/tin-tuc/kinh-te-4/'}
# 'Thế giới': '/vi/tin-the-gioi/', 'Ảnh': '/vi/anh/', 'Video': '/vi/video/', 'Đồ họa': '/vi/graphic/', 'Mega Story': '/vi/megastory/', 'Thông tin nguồn': 'https://news.vnanet.vn/', 'GIỚI THIỆU': '/vi/introl/thong-tan-xa-viet-nam-trung-tam-thong-tin-chien-luoc-tin-cay-cua-dang-va-nha-nuoc-1066.html'
# '

In [18]:
def crawl_data():
    current_time = datetime.datetime.now()
    time_now = current_time.strftime("%Y-%m-%d_%H-%M")
    # print(time_now)
    crawl_thethaovanhoa('raw_thethaovanhoa',category_thethaovanhoa)
    crawl_vietnamplus('raw_vietnamplus', category_vnplus)
    crawl_ttxvn('raw_thongtanxavn',category_ttxvn)

*Merge dữ liệu*

In [19]:
def merge_data(data):
    folder_path = f'./data/raw_data/{data}/'
    csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]
    dataframes = []
    for file in csv_files:
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        dataframes.append(df)
    
    df = pd.concat(dataframes)
    if data != 'thongtanxavn':
        df['news source'] = data
    return df

In [20]:
def create_df_merged():
    df_vietnamplus = merge_data('vietnamplus')
    df_thethaovanhoa = merge_data('thethaovanhoa')
    df_thongtanxavn = merge_data('thongtanxavn')
    df = pd.concat([df_vietnamplus, df_thethaovanhoa, df_thongtanxavn])
    return df

In [21]:
def strip_category(df):
    df['category'] = df['category'].str.strip()
    df['title'] = df['title'].str.strip()
    df['title'] = df['title'].str.lstrip('\n')
    return df

In [22]:
def del_duplicate_links(df):
    duplicate_links = df[df.duplicated('link', keep=False)]
    duplicate_links['categories'] = duplicate_links.groupby('link')['category'].transform(lambda x: ', '.join(x.unique()))

    df_clean = df.drop_duplicates(subset='link', keep='first').copy()

    df_clean = df_clean.merge(duplicate_links[['link', 'categories']], on='link', how='left')

    df_clean['category'] = df_clean['categories'].fillna(df_clean['category'])

    df_clean.drop(columns='categories', inplace=True)

    df_clean = df_clean.drop_duplicates(subset='link', keep='first')

    return df_clean


In [23]:
def del_null_time_data(df):
    df['time'] = df['time'].astype(str)  # Chuyển đổi cột 'time' sang kiểu dữ liệu chuỗi (string)
    df = df.dropna(subset=['time'])  # Loại bỏ các hàng có giá trị NaN hoặc null trong cột 'time'
    return df

In [24]:
def clean_duplicate_news_data(df):
    df_clean = df.drop_duplicates(subset='title', keep='first').copy()
    return df_clean

In [25]:
def convert_time_data(df):
    for index, row in df.iterrows():
        time = row['time']
        df['time'] = df['time'].astype(str)
        if time.endswith("trước"):
            df.at[index, 'time'] = datetime.datetime.now().date()
        elif re.match(r"\d{4}-\d{1,2}-\d{1,2}, \d{1,2}:\d{1,2}", time):
            date = datetime.datetime.strptime(time, "%Y-%m-%d, %H:%M")
            df.at[index, 'time'] = date.strftime("%d/%m/%Y %H:%M")
    
    return df

In [26]:
def remove_infographic_from_title(title):
    if title.startswith("Infographic"):
        title = title[len("Infographic"):].strip()
    return title

In [27]:
def clean_data():
    df = create_df_merged()
    df = df.dropna(subset=['time'])
    df = strip_category(df)
    df = del_duplicate_links(df)
    df = del_null_time_data(df)
    df = clean_duplicate_news_data(df)
    df = convert_time_data(df)
    df['time'] = df['time'].astype(str)  # Chuyển đổi cột 'time' sang kiểu dữ liệu chuỗi (string)
    time_with_dash = df[df['time'].astype(str).str.contains('-')]
    df.loc[time_with_dash.index, 'time'] = df.loc[time_with_dash.index, 'time'].str.split('-').apply(lambda x: f"{x[2]}/{x[1]}/{x[0]}")
    df['title'] = df['title'].apply(lambda x: remove_infographic_from_title(x))
    return df

In [28]:
def export_cleaned_data():
    df = clean_data()
    df.to_csv('data/cleaned_data/cleaned_data.csv', index=False)

In [29]:
# def convert_time_ago(time_ago):
#     if time_ago.endswith("giờ trước"):
#         current_hour = datetime.datetime.now().hour
#         hours_ago = int(time_ago.split()[0])
#         if (hours_ago < current_hour):
#             return datetime.datetime.now().date()
#         else:
#             days = (hours_ago - current_hour)/24
#             return (datetime.datetime.now().date() - datetime.timedelta(days=days))

#     elif re.match(r"\d{1,2} thg \d{1,2}, \d{4}", time_ago):
#         date = datetime.datetime.strptime(time_ago, "%d thg %m, %Y")
#         return date.date()
    
#     else:
#         days_ago = int(time_ago.split()[0])
#         return (datetime.datetime.now().date() - datetime.timedelta(days=days_ago))

In [30]:
# def get_keyword(title):
#     cleaned_text = re.sub(r'^\W+|\W+$', '', title)
#     keyword = cleaned_text.replace(" ", "+")
#     return keyword

#đang sửa dở

In [31]:
# def find_common_keywords(sentence1, sentence2):
#     result1 = ViPosTagger.postagging(ViTokenizer.tokenize(sentence1))
#     result2 = ViPosTagger.postagging(ViTokenizer.tokenize(sentence2))
    
#     words1 = result1[0]
#     tags1 = result1[1]
    
#     words2 = result2[0]
#     tags2 = result2[1]
    
#     keyword_set1 = set([words1[i] for i in range(len(words1)) if tags1[i] in ['N', 'Nc', 'M', 'Np', 'V', 'R']])
#     keyword_set2 = set([words2[i] for i in range(len(words2)) if tags2[i] in ['N', 'Nc', 'M', 'Np', 'V', 'R']])
    
#     common_keywords = keyword_set1.intersection(keyword_set2)
    
#     return len(common_keywords)


In [32]:
# def search_title(first_title, category, time, num_pages=5):
#     keyword = get_keyword(first_title)
#     options = Options()
#     options.add_argument('--dns-prefetch-disable --disable-background-networking --disable-client-side-phishing-detection --disable-default-apps --disable-hang-monitor --disable-popup-blocking --disable-prompt-on-repost --disable-sync --disable-translate --metrics-recording-only --no-first-run --safebrowsing-disable-auto-update --password-store=basic')
#     # --headless 
#     driver = webdriver.Chrome(options=options)

#     num_result = 0
#     time_post = []

#     for page in range(1, num_pages + 1):
#         url = f"https://duckduckgo.com/?q=Nga&va=v&t=ha&ia=web"
#         print(url)
#         driver.get(url)

#         html_content = driver.page_source

#         soup = BeautifulSoup(html_content, 'html.parser')
#         titles_span = soup.find_all('span', class_ = 'EKtkFWMYpwzMKOYr0GYm LQVY1Jpkk8nyJ6HBWKAk')
#         for title in titles_span:
#             print(title.text.strip())

#     #     search_results = soup.find_all('div', class_='react-results--main')

#     #     for result in search_results:
#     #         title = result.find('a', class_='eVNpHGjtxRBq_gLOfGDr LQNqh2U1kzYxREs65IJu').text
#     #         print(title)
#     #         time_ago = result.find('span', class_='result__timestamp').text if result.find('span', class_='result__timestamp') else None
#     #         similarity_ratio = fuzz.ratio(first_title[:45], title[:45])
#     #         print(similarity_ratio)
#     #         if similarity_ratio >= 70:
#     #             if time_ago:
#     #                 time_post.append(convert_time_ago(time_ago))
#     #             num_result += 1
#     #         else:
#     #             description = result.find('a', class_='result__snippet').text
#     #             same_common_keywords = find_common_keywords(first_title, description)
#     #             if same_common_keywords >= 5:
#     #                 if time_ago:
#     #                     time_post.append(convert_time_ago(time_ago))
#     #                 num_result += 1
#     #             elif time_ago:
#     #                 if same_common_keywords >= 3 and convert_time_ago(time_ago) == time[:10]:
#     #                     time_post.append(convert_time_ago(time_ago))
#     #                     num_result += 1

#     # count_dict = {}
#     # for date in time_post:
#     #     if date in count_dict:
#     #         count_dict[date] += 1
#     #     else:
#     #         count_dict[date] = 1

#     # # driver.quit()
#     return category, first_title, num_result, count_dict


In [33]:
# search_title("Họp LHQ: Việt Nam bày tỏ quan ngại về những diễn biến trên Biển Đông", 1, 1)

https://duckduckgo.com/?q=Nga&va=v&t=ha&ia=web
Nga玩家社区 - Nga.cn
NGA玩家社区
Nỗ lực của Ukraine vây hãm lính Nga ở Bakhmut - VnExpress
Nga phản bác tuyên bố của Thổ Nhĩ Kỳ về thỏa thuận ngũ cốc
Tin tức Nga: Tình hình chính trị, quân sự Nga mới nhất hôm nay
NGA Director Frank Whitworth Shares How AI Will Transform GEOINT
Home | National Geospatial-Intelligence Agency
NGA 2023 Annual Meeting - National Governors Association
Tướng Nga ở Ukraine bị sa thải sau khi chỉ trích lãnh đạo
Cuộc Đời Vẫn Đẹp Sao tập 44: Chị Hòa se duyên cho Nga và Thạch
https://duckduckgo.com/?q=Nga&va=v&t=ha&ia=web
Tin tức Nga: Tình hình chính trị, quân sự Nga mới nhất hôm nay
NGA: Tin tức, tình hình Nga và Ukraine mới nhất hôm nay - SOHA
NGA玩家社区
Nga - Wikipedia tiếng Việt
Tin tức Nga: kinh tế, chính trị, quân sự... mới nhất trên VnExpress
Tin tức mới nhất về Nga - SkyDoor
Tình hình Nga và Ukraine hôm nay 14/07: Tại sao Nga muốn chiếm Ukraine?
Báo điện tử tiếng Việt đăng ký tại Nga
Tướng Nga ở Ukraine bị sa thải sau kh

NameError: name 'count_dict' is not defined

In [ ]:
def export_json(start_date, category_in):
    df = pd.read_csv('./data/cleaned_data/cleaned_data.csv')
    titles = df['title']
    
    end_date = datetime.datetime.now().date()

    output_file = f"data/report/{start_date}_{end_date}.json"
    
    with open(output_file, "a", encoding="utf-8") as file:
        count = 0
        for i, title in enumerate(titles):          
            time = df.loc[i, 'time']
            if pd.notnull(time) and isinstance(time, str):
                target_date = datetime.datetime.strptime(time[:10], "%d/%m/%Y").date()
            else:
                continue

            category = df.loc[i, 'category'].strip()
            categories = category.split(", ")
            for category in categories:
                category = category.strip()
            
            # start_date = datetime.datetime.strptime(start_date, '%Y-%m-%d').date()
            if (start_date - target_date).days <= 0 and (category_in in categories or category_in == 'all'):
                category, first_title, num_result, count_dict = search_title(title, df.loc[i, 'category'], df.loc[i, 'time'])
                count_dict_str = {date.strftime('%d/%m/%Y'): count for date, count in count_dict.items()}

                data = {
                    "Số thứ tự": count +1,
                    "Chủ đề": first_title.strip(),
                    "Thời gian đăng bài": df.loc[i, 'time'],
                    "Danh mục": categories,
                    "Số bài đăng": num_result,
                    "Mật độ bài đăng": count_dict_str,
                    "Link bài đăng": df.loc[i, 'link'],
                    "Tóm tắt bài đăng": df.loc[i, 'summary'],
                    "Nội dung": df.loc[i, 'content']
                }
                
                print(data)

                json_data = json.dumps(data, ensure_ascii=False)
                if count > 0:
                    file.write("\n")
                file.write(json_data)
                count += 1
    return start_date

In [ ]:
def update_database():
    crawl_data()
    export_cleaned_data()

In [ ]:
# def export_report(start_date):
#     end_date = datetime.datetime.now().date()
#     # start_date_str = start_date.strftime("%d_%m_%Y")
#     file_path = f"data/report/{start_date}_2023-06-18.json"
    
#     with open(file_path, "r", encoding="utf-8") as file:
#         json_data = file.read()

#     json_strings = json_data.split("\n")

#     data = []
#     for json_string in json_strings:
#         try:
#             item = json.loads(json_string)
#             data.append(item)
#         except json.JSONDecodeError:
#             continue
    
#     for item in data:
#         # num_result = item.get("Số bài đăng", 0)
#         # if num_result == 0:
#         item['Chủ đề'] = remove_infographic_from_title(item['Chủ đề'])
#         category, first_title, item['Số bài đăng'], count_dict= search_title(item['Chủ đề'], item['Danh mục'], item['Thời gian đăng bài'])
#         count_dict_str = {date.strftime('%d/%m/%Y'): count for date, count in count_dict.items()}
#         item['Mật độ bài đăng'] = count_dict_str
#         print(item)
#     json_output = json.dumps(data, ensure_ascii=False, indent=4)

#     # output_file = "data/report/report_all.json"
#     # with open(output_file, "a", encoding="utf-8") as file:
#     #     file.write(json_output)


In [ ]:
def crawl_news_tiktok(news, reader):
    
    m = 0
    
    chrome_options = Options()
    chrome_options.add_argument("--headless")

    driver = webdriver.Chrome(options=chrome_options)
    driver.get(f"https://www.tiktok.com/@{news}")

    driver.implicitly_wait(10)

    full_html = driver.page_source

    soup = BeautifulSoup(full_html, 'html.parser')

    divs = soup.find_all('div', class_='tiktok-x6y88p-DivItemContainerV2 e19c29qe8')

    titles = []
    video_links = []
    video_counts = []
    
    for div in divs:
        title_element = div.find('div', class_='tiktok-16ou6xi-DivTagCardDesc eih2qak1')
        if title_element:
            a_element = title_element.get('aria-label')
            if a_element:
                title = a_element
                titles.append(title)
            else:
                titles.append('')
        else:
            titles.append('')

        img_tag = div.find('img')
        
        image_url = img_tag['src']
        response = requests.get(image_url)
        
        image_path = f'data/images/{news}_{m}.jpg'
        urllib.request.urlretrieve(image_url, image_path)
        m = m + 1
        
        text_list = []
        
        if response.status_code == 200:
            image = Image.open(BytesIO(response.content))
            result = reader.readtext(image)
            
            for text in result:
                text_list.append(text[1])
            
            merged_text = ' '.join(text_list)
            # print(merged_text)
        
        else:
            print(f"Không thể truy cập vào URL: {image_url}")
        video_links.append(merged_text)

        video_count = div.find('strong', class_='video-count').text
        video_counts.append(video_count)
    
    driver.quit()
    data = {'Title': titles, 'Link': video_links, 'Count': video_counts}
    data['Time'] = datetime.datetime.now().strftime("%Y-%m-%d")
    data['Source'] = news
    
    csv_file = 'data/tiktok_data/tiktok_data_test.csv'
    with open(csv_file, 'a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        
        for i in range(len(titles)):
            writer.writerow([titles[i], video_links[i], video_counts[i],  datetime.datetime.now().strftime("%Y-%m-%d"), news])
    
    print(data)
    print(f"Data {news} written to CSV successfully.")

In [ ]:
news = {'vtvcab.giaitri24h', 'vtv24news', 'vtv4go', 'vtvcab.giaitri', 'vtvgo', 'vtvcab.tintuc', 'vietnamplus', 'vietnamnet.vn', 
        'vnexpress.official', 'voh.radio', 'atvtintuc', 'yannews', 'vtv7official', 'vtvlive.tintuc', 'vtv8_daitruyenhinhvn', 'tintuc_vtv123', 
        'vtv3official', 'vtv2life', 'vtvcab.on', 'vnews360', 'thethao247.vn', 'vietnamtopnews.mcv'}

In [ ]:
def daily_update():
    # update_database()
    reader = easyocr.Reader(['en', 'vi'])
    for new in news:
        crawl_news_tiktok(new, reader)

In [ ]:
daily_update()

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


{'Title': ['balan đưa quân đi trấn giữ Biên Cương#CapCut #tintuc #nam2023 #nga #ukraine🇺🇦 #vba2023', '#nga #ukraine🇺🇦 #thegioi #nam2023 #tinmoi #CapCut #tintuc', '#tinmoi #nam2023 #nga #ukraine🇺🇦 #thegioi', '#vba2023 #CapCut #tinmoi #nga #ukraine🇺🇦 #nam2023', '#vba2023 #tinmoi #nam2023 #nga #ukraine🇺🇦 #CapCut', '#vba2023 #tintuc #ukraine🇺🇦 #nga #nam2023 #CapCut #tinmoi', '#CapCut #tinmoi #nam2023 #nga #ukraine🇺🇦 #vba2023 #tintuc', 'bình luận Nga và Ukraine#tintuc #tinmoi #nga #ukraine🇺🇦 #nam2023 #CapCut', '#CapCut #vietnam #trungquoc', '#tinmoi #nga #ukraine🇺🇦 #nam2023 #thegioi #phuongtay#nato', '#nga #ukraine🇺🇦 #FoodFestOnTikTok #tinmoi', '#tintuc #tinmoi #nga #FoodFestOnTikTok #ukraine🇺🇦', '', '#CapCut #tinmoi #tintuc #nga #ukraine🇺🇦 #TryItWithTikTok #thegioi', '#CapCut #tinmoi #tintuc #ukraine🇺🇦 #nga #TryItWithTikTok #my', 'nói chung là Ukraine ko muốn dừng chiến tranh với Nga #CapCut #TryItWithTikTok #tinmoi #tintuc #ukraine🇺🇦 #nga #LearnOnTikTok', '#CapCut #ukraine🇺🇦 #nga #tintuc 

In [ ]:
df = pd.read_csv('data/tiktok_data/tiktok_data.csv')
df = df[::-1]
df.drop_duplicates(subset='Link', keep='first', inplace=True)
df.drop_duplicates(subset=['Title', 'Source'], keep='first', inplace=True)
df.head(50)

,Title,Link,Count,Time,Source
3317,NaN,https://p16-sign-sg.tiktokcdn.com/tos-alisg-p-...,11K,2023-06-30,vtv4go
3287,NaN,https://p16-sign-sg.tiktokcdn.com/tos-alisg-p-...,3368,2023-06-30,vietnamplus
3257,NaN,https://p9-sign-sg.tiktokcdn.com/tos-alisg-p-0...,25.9K,2023-06-30,vietnamnet.vn
3227,NaN,https://p16-sign-va.tiktokcdn.com/tos-useast2a...,7.7M,2023-06-30,tintuc60giaymoingay
3215,NaN,https://p16-sign-sg.tiktokcdn.com/tos-alisg-p-...,298.6K,2023-06-30,vtvgo
3185,NaN,https://p16-sign-sg.tiktokcdn.com/tos-alisg-p-...,93.9K,2023-06-30,vtvcab.tintuc
3155,NaN,https://p16-sign-va.tiktokcdn.com/tos-maliva-p...,8411,2023-06-30,kenh14disoisaodi
3125,NaN,https://p16-sign-sg.tiktokcdn.com/tos-alisg-p-...,14K,2023-06-30,vnexpress.official
3095,NaN,https://p16-sign-sg.tiktokcdn.com/tos-alisg-p-...,1072,2023-06-30,vnews360
3066,NaN,https://p9-sign-sg.tiktokcdn.com/tos-alisg-p-0...,2041,2023-06-30,vtvlive.tintuc


In [ ]:
def convert_count_to_number(count_str):
    count_str = count_str.replace(',', '')  # Xóa dấu phẩy phân cách nghìn (nếu có)
    if 'K' in count_str:
        count_str = count_str.replace('K', '')
        count_float = float(count_str) * 1000
        return int(count_float)
    elif 'M' in count_str:
        count_str = count_str.replace('M', '')
        count_float = float(count_str) * 1000000
        return int(count_float)
    else:
        return int(count_str)
    
# Áp dụng hàm chuyển đổi cho cột "count" sử dụng phương thức apply
df['Count'] = df['Count'].apply(convert_count_to_number)

# Hiển thị 10 dòng đầu tiên của DataFrame sau khi chuyển đổi
df.head(10)

,Title,Link,Count,Time,Source
3317,NaN,https://p16-sign-sg.tiktokcdn.com/tos-alisg-p-...,11000,2023-06-30,vtv4go
3287,NaN,https://p16-sign-sg.tiktokcdn.com/tos-alisg-p-...,3368,2023-06-30,vietnamplus
3257,NaN,https://p9-sign-sg.tiktokcdn.com/tos-alisg-p-0...,25900,2023-06-30,vietnamnet.vn
3227,NaN,https://p16-sign-va.tiktokcdn.com/tos-useast2a...,7700000,2023-06-30,tintuc60giaymoingay
3215,NaN,https://p16-sign-sg.tiktokcdn.com/tos-alisg-p-...,298600,2023-06-30,vtvgo
3185,NaN,https://p16-sign-sg.tiktokcdn.com/tos-alisg-p-...,93900,2023-06-30,vtvcab.tintuc
3155,NaN,https://p16-sign-va.tiktokcdn.com/tos-maliva-p...,8411,2023-06-30,kenh14disoisaodi
3125,NaN,https://p16-sign-sg.tiktokcdn.com/tos-alisg-p-...,14000,2023-06-30,vnexpress.official
3095,NaN,https://p16-sign-sg.tiktokcdn.com/tos-alisg-p-...,1072,2023-06-30,vnews360
3066,NaN,https://p9-sign-sg.tiktokcdn.com/tos-alisg-p-0...,2041,2023-06-30,vtvlive.tintuc


In [ ]:
df_sorted = df.sort_values('Count', ascending=False)
df_sorted.head(20)

,Title,Link,Count,Time,Source
1179,Người chồng lo cho vợ nằm viện đến nỗi bán hết...,https://p16-sign-va.tiktokcdn.com/tos-useast2a...,8199999,2023-06-27,theanh28.tv
1573,Đâu mới là khuôn mặt thật sự của anh vậy??? #v...,https://p16-sign-sg.tiktokcdn.com/tos-alisg-p-...,8000000,2023-06-27,vtvcab.giaitri
3227,NaN,https://p16-sign-va.tiktokcdn.com/tos-useast2a...,7700000,2023-06-30,tintuc60giaymoingay
1260,Phần 14| Trọng tài đã phải ôm mặt khi chứng ki...,https://p16-sign-va.tiktokcdn.com/tos-useast2a...,7600000,2023-06-27,tintuc60giaymoingay
1256,Người vợ tâm lí nhất quả đất!#xuhuong #tinmoiv...,https://p16-sign-va.tiktokcdn.com/tos-useast2a...,6100000,2023-06-27,tintuc60giaymoingay
1258,"""Mẹ và em ở nhà nha, con xin lỗi"" thanh niên 2...",https://p16-sign-va.tiktokcdn.com/tos-useast2a...,5700000,2023-06-27,tintuc60giaymoingay
1187,Ngoài đời gặp ông chủ vậy là căng áh #theanh28,https://p16-sign-va.tiktokcdn.com/tos-useast2a...,5500000,2023-06-27,theanh28.tv
1174,Nếu bạn có người mẹ như vậy thì bạn có cảm thấ...,https://p16-sign-va.tiktokcdn.com/tos-useast2a...,5500000,2023-06-27,theanh28.tv
1175,Đi tiếp hay dừng lại là quyết định của mỗi ngư...,https://p16-sign-va.tiktokcdn.com/tos-useast2a...,5000000,2023-06-27,theanh28.tv
1249,Phải ngã bảo nhiêu lần để có được kĩ năng vừa ...,https://p16-sign-va.tiktokcdn.com/tos-useast2a...,4500000,2023-06-27,tintuc60giaymoingay


In [ ]:
def find_common_keywords(sentence):
    if isinstance(sentence, float):
        return ''
    result = ViPosTagger.postagging(ViTokenizer.tokenize(sentence))
    words = result[0]
    tags = result[1]
    keyword_set = set([words[i].replace('_', ' ') for i in range(len(words)) if tags[i] in ['N', 'Nc', 'M', 'Np', 'V', 'R']])
    return keyword_set

In [ ]:
df['Title'] = df['Title'].apply(lambda title: ' '.join(word for word in str(title).split() if not word.startswith('#')))

df['Keyword'] = df['Title'].apply(find_common_keywords)

df.to_csv("data_test.csv", index=False)

In [ ]:
df_cleaned = pd.read_csv("data/cleaned_data/cleaned_data.csv")

df_cleaned['keyword'] = df_cleaned['title'].apply(find_common_keywords)

df_cleaned['time'] = df_cleaned['time'].apply(lambda x: pd.to_datetime(x[:10], format='%d/%m/%Y'))

df_cleaned = df_cleaned.sort_values(by='time', ascending=False)

df_cleaned.to_csv("cleaned_data_test.csv", index=False)

In [ ]:
df_news = pd.read_csv("cleaned_data_test.csv")
df_news['Count'] = 0
df_news['Source'] = ""

df_tiktok = pd.read_csv("data_test.csv")

for i in range(len(df_news)):
    current_row = df_news.loc[i]
    current_keywords = current_row['keyword']
    # print(current_keywords)
    keyword_list = current_keywords.split(',')
    matching_rows = []

    if len(keyword_list) <= 1:
        continue
    
    # print(f"Hàng chính: {current_row.name}: {current_row['title']}")
    for j in range(len(df_tiktok)):
        compare_row = df_tiktok.loc[j]
        compare_keywords = compare_row['Keyword'].split(',')  # Thay đổi thành danh sách từ khóa

        matching_count = sum(1 for keyword in keyword_list if keyword in compare_keywords)
        # if matching_count >= 6:
        #     print(f"----- Hàng {compare_row.name}: {compare_row['Title']}: {compare_row['Count']}: {compare_row['Source']}")

        if matching_count > 0.5 * len(keyword_list) and matching_count > 0.5 * len(compare_keywords):
            matching_rows.append(compare_row)
            # rows_to_drop.append(j)

    if len(matching_rows) > 0:
        for row in matching_rows:
            current_row['Source'] += ", " + row['Source']
            current_row['Count'] += row['Count']
            print(f"Hàng {row.name}: {row['Title']}: {row['Count']}: {row['Source']}")
        # print(keyword_list, "------", compare_keywords)  # Sửa thành keyword_list và compare_keywords
        print(f"Hàng chính: {current_row.name}: {current_row['title']}: {current_row['Count']}: {current_row['news source']}")
        print()
        print()


Hàng 95: Thủ tướng Trung Quốc Lý Cường chủ trì lễ đón Thủ tướng Phạm Minh Chính: 22200: vietnamnet.vn
Hàng chính: 284: Thủ tướng Lý Cường chủ trì Lễ đón Thủ tướng Phạm Minh Chính thăm chính thức Trung Quốc: 22200: Baotintuc.vn


Hàng 529: Mưa lớn khiến nhiều tuyến đường, hoa màu ngập sâu: 111: vtvlive.tintuc
Hàng chính: 367: Mưa kéo dài tại TP Hồ Chí Minh, nhiều tuyến đường ngập nặng: 111: vietnamplus


Hàng 192: Hôm nay (27/6) hơn 1 triệu thí sinh làm thủ tục thi tốt nghiệp THPT 2023: 2757: voh.radio
Hàng chính: 647: Kỳ thi tốt nghiệp THPT 2023: Hơn 1 triệu thí sinh làm thủ tục dự thi: 2757: thethaovanhoa


Hàng 412: Công an Long An khẩn trương truy tìm 3 luật sư vụ Tịnh thất Bồng lai: 864: vietnamplus
Hàng chính: 669: Bản tin 60s: Khẩn trương truy tìm 3 luật sư vụ Tịnh thất Bồng lai: 864: vietnamplus


Hàng 500: Xét xử vụ tham ô tại Bộ Tư lệnh Cảnh sát biển Việt Nam: 5317: vnews360
Hàng chính: 780: Ngày 27/6, xét xử vụ tham ô tại Bộ Tư lệnh Cảnh sát biển Việt Nam: 5317: vietnamplus



In [ ]:

rows_to_drop = []

df = df.reset_index(drop=True)

for i in range(len(df)):
    current_row = df.loc[i]
    current_keywords = set(current_row['Keyword'])
    matching_rows = []
    # print(len(current_keywords))
    if len(current_keywords) <= 1:
        continue
    
    for j in range(i+1, len(df)):
        if j in df.index:
            compare_row = df.loc[j]
            compare_keywords = compare_row['Keyword']
            
            matching_count = sum(1 for keyword in current_keywords if keyword in compare_keywords)
            if matching_count >= 0.5 * len(current_keywords) and matching_count >= 0.5 * len(compare_keywords) and current_row['Source'] != compare_row['Source'] :
                matching_rows.append(compare_row)
                rows_to_drop.append(j)
            
    if len(matching_rows) > 0:
        print(f"Hàng {current_row.name}: {current_row['Title']}: {current_row['Count']}: {current_row['Source']}")
        for row in matching_rows:
            current_row['Source'] += ", " + row['Source']
            current_row['Count'] += row['Count']
            print(f"Hàng {row.name}: {row['Title']}: {row['Count']}: {row['Source']}")
        print()

Hàng 37: Người tàn ác thì sống thảnh thơi :)): 63700: vtvcab.giaitri
Hàng 206: Người tàn ác thường sống thảnh thơi: 12000: yeah1.gnews

Hàng 74: Lực lượng chức năng tỉnh Kon Tum cho biết vừa bắt giữ 2 đối tượng có hành vi vận chuyển 34 bánh heroin từ Lào về Việt Nam: 275800: vtv24news
Hàng 251: Lực lượng chức năng tỉnh Kon Tum cho biết vừa bắt giữ 2 đối tượng có hành vi vận chuyển 34 bánh heroin từ Lào về Việt Nam#vtvcabtintuc#vtvcab#tiktoknews: 83000: vtvcab.tintuc

Hàng 89: Hình ảnh 100.000 cây xanh hai bên cao tốc Cầu Giẽ - Ninh Bình đang bị đốn hạ: 264100: vietnamnet.vn
Hàng 408: 100.000 cây xanh hai bên cao tốc Cầu Giẽ - Ninh Bình đang bị đốn hạ: 24700: vietnamplus

Hàng 110: Phá đường dây mua bán thiết bị gian lận thi cử trước kỳ thi tốt nghiệp: 21300: vietnamnet.vn
Hàng 191: Phá 2 đường dây thiết bị gian lận thi cử siêu nhỏ: 4622: voh.radio

Hàng 111: Cục sạc dự phòng phát nổ trong phòng trọ ở Bình Dương: 1842: vietnamnet.vn
Hàng 253: Cục sạc dự phòng phát nổ như bom trong dãy t